In [121]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time
import json
import requests

from bs4 import BeautifulSoup

## Functions

In [319]:
def get_urls(driver):
    base_url = "https://jamalon.com/en/best-seller-books-2019?p={}&___from_store=ar"
    book_urls = []
    for i in range(1, 6):
        driver.get(base_url.format(i))

        books = driver.find_elements_by_class_name("book")
        for book in books:
            book_urls.append(book.find_element_by_tag_name("a").get_attribute("href"))
        
        time.sleep(0.5)
    return book_urls

def get_cookie(driver, url):
    driver.get(url)
    title = driver.find_element_by_xpath('//*[@id="content"]/section/div/div/div[2]/div[1]/div/div[1]').text
    sku = driver.find_element_by_xpath('//*[@id="content"]/section/div/div/div[2]/div[2]/div[2]/div[1]/div[2]/ul/li[2]').text
    isbn = driver.find_element_by_xpath('//*[@id="content"]/section/div/div/div[2]/div[2]/div[2]/div[1]/div[2]/ul/li[5]').text
    
    data[title] = {}
    data[title]["sku"] = sku
    data[title]["isbn"] = isbn
    
    add_to_cart_button = driver.find_element_by_class_name("btn-cart")
    driver.execute_script("arguments[0].scrollIntoView();", add_to_cart_button)
    add_to_cart_button.click()
    
    return {
        "currency": "USD",
        "currency_code": "VVNE",
        "frontend": driver.get_cookie("frontend")["value"],
        "__utmt": "1",
        "store": "en"
    }

def clear_shopping_cart():
    driver.get('https://jamalon.com/en/checkout/cart/')
    items_to_remove = driver.find_elements_by_xpath('//a[@title="Remove item"]')
    for item in items_to_remove:
        item.click()
        driver.switch_to.alert.accept()
    
    time.sleep(2)

## Main

In [325]:
driver = webdriver.Firefox()

### Get URLs

In [324]:
urls = get_urls(driver)
urls = list(map(lambda x: x.replace('/ar/', '/en/'), urls))

KeyboardInterrupt: 

### Crawl using requests

In [424]:
def get_top_books():
    num_pages = 5
    base_url = "https://jamalon.com/en/best-seller-books-2019/?p={}"
    
    books_info = {}
    
    for i in range(1, num_pages+1):
        response = requests.get(base_url.format(i), cookies={
            "currency": "USD",
            "currency_code": "VVNE"
        })
        
        html = BeautifulSoup(response.content)
        books = html.find_all('a', attrs={'class': 'product-image'})
        for book in books:
            title = book['title']
            url = book['href']
            price = book.find('div', attrs={'class', 'new-price'}).text.strip()

            books_info[title] = {}
            books_info[title]['url'] = url
            books_info['price'] = price
    
    return books_info

In [425]:
books_info = get_top_books()

In [426]:
books_info

{'القوقعة : يوميات متلصص': {'url': 'https://jamalon.com/ar/1004376.html'},
 'price': '$22.26',
 'أولاد حارتنا': {'url': 'https://jamalon.com/ar/1006424.html'},
 'الرجال من المريخ النساء من الزهرة': {'url': 'https://jamalon.com/ar/1006434.html'},
 'الجريمة والعقاب 1/2 : ترجمة زينب المزديوي': {'url': 'https://jamalon.com/ar/1007803.html'},
 'الأب الغني الأب الفقير': {'url': 'https://jamalon.com/ar/1014741.html'},
 'رواية 1984': {'url': 'https://jamalon.com/ar/3059915.html'},
 'التخلف الاجتماعي : مدخل إلى سيكولوجية الإنسان المقهور': {'url': 'https://jamalon.com/ar/3059871.html'},
 'الخيميائي': {'url': 'https://jamalon.com/ar/3033435.html'},
 'السياسة والحيلة عند العرب : رقائق الحلل في دقائق الحيل': {'url': 'https://jamalon.com/ar/3001866.html'},
 'العصفورية': {'url': 'https://jamalon.com/ar/3026103.html'},
 'حياة في الإدارة': {'url': 'https://jamalon.com/ar/3019715.html'},
 'خوارق اللاشعور : أو أسرار الشخصية الناجحة': {'url': 'https://jamalon.com/ar/3016259.html'},
 'رسائل غسان كنفاني إلى

In [326]:
a = get_cookie(driver, urls[0])

In [408]:
books

{'القوقعة : يوميات متلصص': {'url': 'https://jamalon.com/ar/1004376.html'},
 'price': '$12.65'}

In [327]:
a

{'currency': 'USD',
 'currency_code': 'VVNE',
 'frontend': 'h4vehamjer17e205ultu932d67',
 '__utmt': '1',
 'store': 'en'}

In [ ]:
response = requests.post(request_url, cookies=a)
content = response.json()
parsed_html = BeautifulSoup(content['summary'])
title = parsed_html.body.find('td', attrs={'class': 'name'}).text.strip()
prices = list(map(lambda x: x.text.strip(), parsed_html.body.find_all('span', attrs={'class': 'price'})))

print(title)
print(prices)

In [316]:
response.cookies

<RequestsCookieJar[Cookie(version=0, name='frontend', value='tvfcio9iv3pfvh50k3jv1p9mc2', port=None, port_specified=False, domain='.jamalon.com', domain_specified=True, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=1607700609, discard=False, comment=None, comment_url=None, rest={}, rfc2109=False)]>

In [313]:
parsed_html.body

<body><table class="onestepcheckout-summary">
<thead>
<tr>
<th class="name">الكتاب</th>
<th class="qty">الكمية</th>
<th class="total">المجموع الفرعي</th>
</tr>
</thead>
<tr>
<td class="name">
                الأب الغني الأب الفقير                            </td>
<td class="qty">3</td>
<td class="total">
<span class="price">OMR17.31</span> </td>
</tr>
<tr>
<td class="name">
                القوقعة : يوميات متلصص                            </td>
<td class="qty">4</td>
<td class="total">
<span class="price">OMR19.48</span> </td>
</tr>
</table>
<table class="onestepcheckout-totals">
<tr>
<td class="a-right cart-points-total_spent" colspan="1">
<div class="">النقاط المستهلكة :</div>
</td>
<td class="a-right cart-points-total_spent">
<div class="points-amount">
                0 نقاط            </div>
</td>
</tr>
<tr>
<td class="a-right cart-points-total_spent cart-points-spending_warnings" colspan="2">
</td>
</tr>
<tr>
<td class="a-right cart-points-total_earned" colspan="1">
<div class=""

In [312]:
def number_span(tag):
    return tag.name=='span' and 'Shipping' in tag.parent.contents[0]

parsed_html.body.find_all(number_span)

[]

In [96]:
request_url = "https://jamalon.com/en/onestepcheckout/ajax/save_billing"

In [82]:
data = {
    "url": "https://jamalon.com/en/1004376.html",
    "user_id": "ed7b1018-1bf1-5282-a9b7-5e457b5ca52d",
    "window_height": 731,
    "window_width": 1920
}


In [113]:
all_cookies = driver.get_cookies()
cookies_to_send = {}
for c in all_cookies:
    cookies_to_send[c['name']] = c['value']

In [142]:
temp = cookies_to_send.copy()

In [253]:
temp.pop('frontend')

'tvfcio9iv3pfvh50k3jv1p9mc2'

In [254]:
temp

{'_hjAbsoluteSessionInProgress': '0',
 '_hjFirstSeen': '1',
 '_hjIncludedInPageviewSample': '1',
 '_hjIncludedInSessionSample': '1'}

In [290]:
temp['frontend'] = 'tvfcio9iv3pfvh50k3jv1p9mc2'

In [291]:
temp

{'_hjAbsoluteSessionInProgress': '0',
 '_hjFirstSeen': '1',
 '_hjIncludedInPageviewSample': '1',
 '_hjIncludedInSessionSample': '1',
 'frontend': 'tvfcio9iv3pfvh50k3jv1p9mc2'}

In [292]:
response = requests.post(request_url, cookies=temp).json()
print(response)
parsed_html = BeautifulSoup(response['summary'])
title = parsed_html.body.find('td', attrs={'class': 'name'}).text.strip()
prices = list(map(lambda x: x.text.strip(), parsed_html.body.find_all('span', attrs={'class': 'price'})))

print(title)
print(prices)

{'shipping_method': '\n            <dl class="shipment-methods">\n                                    <!--<dd></dd>-->\n                                            <dt style="margin-bottom: 5px;">\n\n                                                           <input name="shipping_method" type="radio" class="validate-one-required-by-name" value="shipmanager_shipmanager_7" id="s_method_shipmanager_shipmanager_7" checked="checked" />\n                                <label for="s_method_shipmanager_shipmanager_7"><!--<b>Available Shipping Methods:</b>--> Express Shipping - (7-14 days)                                <strong>\n                                                                \n                                <span class="price">OMR0.00</span>                                                                </strong>\n                                </label>\n                                                   </dt>\n                                        </dl>\n    ', 'payment_

In [293]:
a = requests.post('https://jamalon.com/en/checkout/cart/add/uenc/aHR0cHM6Ly9qYW1hbG9uLmNvbS9lbi8xMDA2NDI0Lmh0bWw_X19fU0lEPVU,/product/2972/')


In [283]:
a.cookies

<RequestsCookieJar[Cookie(version=0, name='frontend', value='4d533mmvp086lu2qh30nn26917', port=None, port_specified=False, domain='.jamalon.com', domain_specified=True, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=1607699754, discard=False, comment=None, comment_url=None, rest={}, rfc2109=False), Cookie(version=0, name='currency_code', value='T01S', port=None, port_specified=False, domain='.jamalon.com', domain_specified=True, domain_initial_dot=False, path='/ar', path_specified=True, secure=False, expires=1639149354, discard=False, comment=None, comment_url=None, rest={}, rfc2109=False), Cookie(version=0, name='currency_code', value='T01S', port=None, port_specified=False, domain='.jamalon.com', domain_specified=True, domain_initial_dot=False, path='/en', path_specified=True, secure=False, expires=1639149354, discard=False, comment=None, comment_url=None, rest={}, rfc2109=False)]>

In [204]:
# r = requests.post('https://in.hotjar.com/api/v2/client/sites/1584878/visit-data?sv=7', json=data)
# r.content

### Crawl using RPA -- slow

In [5]:
data = {}
for url in urls[1:]:
    driver.get(url)
    
    title = driver.find_element_by_xpath('//*[@id="content"]/section/div/div/div[2]/div[1]/div/div[1]').text
    sku = driver.find_element_by_xpath('//*[@id="content"]/section/div/div/div[2]/div[2]/div[2]/div[1]/div[2]/ul/li[2]').text
    isbn = driver.find_element_by_xpath('//*[@id="content"]/section/div/div/div[2]/div[2]/div[2]/div[1]/div[2]/ul/li[5]').text
    
    data[title] = {}
    data[title]["sku"] = sku
    data[title]["isbn"] = isbn
    
    add_to_cart_button = driver.find_element_by_class_name("btn-cart")
    driver.execute_script("arguments[0].scrollIntoView();", add_to_cart_button)
    add_to_cart_button.click()

    time.sleep(1)

    checkout_button = driver.find_elements_by_class_name("btn-checkout")[-1]
    driver.execute_script("arguments[0].scrollIntoView();", checkout_button)
    checkout_button.click()
    
    time.sleep(0.5)

    countries = driver.find_element_by_id("billing:country_id").find_elements_by_tag_name("option")
    countries = list(filter(lambda x: x.text != '', countries))
    
    data[title]["price"] = {}
    
    time.sleep(1)

    for country in countries:
        try:
            country.click()
            time.sleep(0.5)

            totals = driver.find_element_by_class_name("onestepcheckout-totals").find_elements_by_tag_name('tr')
            data[title]["price"][country.text] = {}
            for i in range(1, 4):
                totals_info = totals[-i].find_elements_by_tag_name("td")
                data[title]["price"][country.text][totals_info[0].text] = totals_info[1].text
        except NoSuchElementException:
            pass

        
    clear_shopping_cart()
    
    with open('data.json', 'w') as f:
        json.dump(data, f)



IndexError: list index out of range

In [7]:
with open('data.json', 'w') as f:
    json.dump(data, f)